# Modelos de Regresión Logística
Repo: https://github.com/Diegoval-Dev/DM-HDT7

- Gerson Ramirez - 22281
- Diego Valenzuela - 22309

## Paso 1: Crear variables dicotómicas para las categorías de precio

Para aplicar modelos de clasificación binaria como la regresión logística, es necesario transformar la variable categórica que representa el precio de la vivienda en variables dicotómicas. En ejercicios anteriores se clasificaron las viviendas en tres categorías: `cara`, `media` y `económica`.

En este paso, crearemos tres nuevas columnas binarias:
- `cara_o_no`: toma valor 1 si la vivienda es cara, 0 en caso contrario.
- `media_o_no`: toma valor 1 si la vivienda es de precio medio, 0 en caso contrario.
- `economica_o_no`: toma valor 1 si la vivienda es económica, 0 en caso contrario.

Estas variables permitirán construir modelos independientes para predecir cada categoría.


In [4]:
import pandas as pd

df = pd.read_csv("processed_data.csv")

print("Categorías únicas:", df['PriceCategory'].unique())

df['cara_o_no'] = (df['PriceCategory'] == 'Cara').astype(int)
df['media_o_no'] = (df['PriceCategory'] == 'Intermedia').astype(int)
df['economica_o_no'] = (df['PriceCategory'] == 'Economica').astype(int)

df[['PriceCategory', 'cara_o_no', 'media_o_no', 'economica_o_no']].head()



Categorías únicas: ['Cara' 'Intermedia' 'Economica']


,PriceCategory,cara_o_no,media_o_no,economica_o_no
0,Cara,1,0,0
1,Intermedia,0,1,0
2,Cara,1,0,0
3,Intermedia,0,1,0
4,Cara,1,0,0


### Análisis del Paso 1: Variables dicotómicas

Las nuevas columnas `cara_o_no`, `media_o_no` y `economica_o_no` fueron creadas exitosamente a partir de la variable categórica `PriceCategory`. Cada una representa una clase como una variable binaria (1 si pertenece a esa clase, 0 si no).

Este paso es esencial para aplicar modelos de clasificación binaria como la regresión logística. Al usar estas variables, se podrá entrenar un modelo independiente para predecir si una vivienda pertenece o no a cada una de las categorías definidas.

El resultado obtenido muestra que las variables se asignaron correctamente. Por ejemplo, una vivienda con `PriceCategory = "Cara"` tiene `cara_o_no = 1` y `media_o_no` y `economica_o_no` iguales a 0.


## Paso 2: Cargar los mismos conjuntos de entrenamiento y prueba utilizados anteriormente

Para mantener la consistencia con los análisis previos y permitir comparaciones justas entre modelos, utilizaremos los mismos conjuntos de entrenamiento y prueba que se usaron en ejercicios anteriores como KNN y Naive Bayes.

Estos conjuntos fueron generados a partir del dataset procesado, utilizando una división del 80% para entrenamiento y 20% para prueba, con una semilla (`random_state`) fija para garantizar reproducibilidad. En esta hoja se reutilizará esa misma división, filtrando además solo las columnas necesarias para el modelo actual.


In [5]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['PriceCategory', 'cara_o_no', 'media_o_no', 'economica_o_no'])

y = df['cara_o_no']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Tamaño X_train:", X_train.shape)
print("Tamaño X_test:", X_test.shape)
print("Distribución en y_train:", y_train.value_counts(normalize=True))
print("Distribución en y_test:", y_test.value_counts(normalize=True))


Tamaño X_train: (1168, 76)
Tamaño X_test: (292, 76)
Distribución en y_train: 0    0.669521
1    0.330479
Name: cara_o_no, dtype: float64
Distribución en y_test: 0    0.667808
1    0.332192
Name: cara_o_no, dtype: float64


### Análisis del Paso 2: División en conjuntos de entrenamiento y prueba

Se utilizó la función `train_test_split` para dividir el conjunto de datos en entrenamiento (80%) y prueba (20%), manteniendo la proporción de clases mediante el parámetro `stratify=y`.

Se utilizó como variable objetivo la columna `cara_o_no`, que representa si una vivienda es cara o no. La matriz `X` contiene únicamente las variables predictoras, excluyendo las etiquetas categóricas y las variables dummy adicionales.

Los tamaños de los conjuntos son:
- Entrenamiento: 1168 muestras
- Prueba: 292 muestras

La distribución de clases se conserva adecuadamente:
- Entrenamiento: 33% viviendas caras, 67% no caras
- Prueba: 33% viviendas caras, 67% no caras

Esta división permitirá entrenar y evaluar el modelo de forma confiable y reproducible.


## Paso 3: Entrenar un modelo de regresión logística (cara vs no cara)

En este paso se entrenará un modelo de regresión logística para predecir si una vivienda pertenece a la categoría `Cara` o no. Para ello, se utilizará como variable dependiente la columna `cara_o_no` y como variables independientes el conjunto de predictores previamente definido.

Se aplicará validación cruzada con 5 particiones (`cv=5`) para obtener métricas de desempeño estables. Además, se calcularán los coeficientes del modelo para analizar la influencia de cada variable en la probabilidad de que una vivienda sea clasificada como `Cara`.

Este modelo servirá como base para evaluar qué tan bien se puede predecir esta categoría utilizando regresión logística.


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

model = LogisticRegression(max_iter=1000, solver='liblinear')

cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

model.fit(X_train, y_train)

print("Accuracy promedio (validación cruzada):", round(np.mean(cv_scores), 4))
print("Desviación estándar:", round(np.std(cv_scores), 4))

coef_df = pd.DataFrame({
    'Variable': X_train.columns,
    'Coeficiente': model.coef_[0]
}).sort_values(by='Coeficiente', key=abs, ascending=False)

coef_df.head(10)


Accuracy promedio (validación cruzada): 0.9949
Desviación estándar: 0.0032


,Variable,Coeficiente
72,YrSold,-0.103332
19,YearRemodAdd,-0.090081
57,GarageYrBlt,-0.056699
18,YearBuilt,-0.051794
3,LotFrontage,-0.018195
68,ScreenPorch,-0.017095
1,MSSubClass,0.015714
66,EnclosedPorch,-0.013813
44,LowQualFinSF,0.012296
65,OpenPorchSF,-0.011958


### Análisis del Paso 3: Modelo de regresión logística (cara vs no cara)

Se entrenó un modelo de regresión logística para predecir si una vivienda pertenece a la categoría `Cara` o no. El modelo fue evaluado utilizando validación cruzada con 5 particiones (`cv=5`) sobre el conjunto de entrenamiento.

#### Resultados:
- **Accuracy promedio (CV):** 0.9949
- **Desviación estándar:** 0.0032

Estos valores indican un rendimiento extremadamente alto del modelo en la etapa de entrenamiento, lo cual podría ser una señal de **sobreajuste**, por lo que será importante compararlo con el rendimiento en el conjunto de prueba más adelante.

#### Coeficientes más relevantes del modelo:
Los siguientes predictores fueron los que más influyeron (en valor absoluto) en la clasificación:

| Variable         | Coeficiente  |
|------------------|--------------|
| YrSold           | -0.1033      |
| YearRemodAdd     | -0.0901      |
| GarageYrBlt      | -0.0567      |
| YearBuilt        | -0.0518      |
| LotFrontage      | -0.0182      |
| ScreenPorch      | -0.0171      |

Los coeficientes negativos indican que a mayor valor de esa variable, menor probabilidad de que la vivienda sea clasificada como `Cara`. Por ejemplo, casas más nuevas (`YrSold`, `YearBuilt`, `YearRemodAdd`) tienden a no ser clasificadas como `Cara`, posiblemente porque la categoría `Cara` está asociada con otras características de lujo más que con la antigüedad.

Este modelo servirá como base para evaluar el sobreajuste, la capacidad predictiva y para compararlo con otros clasificadores más adelante.
